In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import datetime as dt
from scipy import stats
from sklearn import preprocessing
import matplotlib as mpl
from scipy.stats import skew
import geopandas
import seaborn as sns

## Plotting parameters
mpl.rcParams['pdf.fonttype'] = 42

## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466

In [92]:
def readModelFile(i):
    try:
        df = pd.read_csv('../data/pnwNP_modeledData_NWM3/'+str(pnwNP['gage'][i])+".csv")
        df['time'] = pd.to_datetime(df['time'])
       
    ### Deliniate time periods
        df['climate_year'] = df.time.dt.year.where(df.time.dt.month < 4, df.time.dt.year + 1)
        df.loc[(df.time.dt.month > 7) & (df.time.dt.month < 10), "flow_season"] = "late_summer"
        df.loc[(df.time.dt.month > 2) & (df.time.dt.month < 6), "season"] = "spring"
        df.loc[(df.time.dt.month > 5) & (df.time.dt.month < 9), "season"] = "summer"
        df.loc[(df.time.dt.month > 8) & (df.time.dt.month < 12), "season"] = "fall"
        df.loc[(df.time.dt.month > 11), "season"]  = "winter"
        df.loc[(df.time.dt.month < 3), "season"] = "winter"
        df['month'] = df.time.dt.month
        df["DOY"] = df.time.dt.day_of_year
        
        # df = df.drop(['Unnamed: 0.1'],axis=1)
        df = df[df["streamflow_NWIS"].notna()].reset_index(drop=True)
        return df

    except:
        pass

In [93]:
def calcZeroFlow(df):
    import warnings
    warnings.filterwarnings("ignore")
    '''
    Read in data files for gages and calculate number of late-summer low flow days
    '''
    

    stat = "ZeroFlow"
    columns = ["gage","VIC_"+stat,"PRMS_"+stat,"NWM2d0_"+stat,"NWM2d1_"+stat,"NWM3"+stat,"NWIS_"+stat]
    out = []

    for column in df[["streamflow_VIC","streamflow_PRMS",'streamflow_NWM2d0','streamflow_NWM2d1','streamflow_NWM3','streamflow_NWIS']]:
        
        #### Round columns to nearest 0.1 cfs, therefore anything less than 0.05 cfs is q=0
        #### Round columns to add zeros
        if column == "streamflow_NWIS":
            df[column] = df[column]/cfs_2_cms
            df[column] = np.round(df[column],1)
            df.loc[df[column]==0,"new"] = 1
        else:
            df.loc[df[column]<=(df[column].mean()*.1),"new"] = 1

        ## Calculate number of zero flow days by season
        lsq10d = df[['climate_year','new']]
        if df["new"].isnull().all():
            lsq10d = np.nan
        else:
            lsq10d = lsq10d.groupby(["climate_year"]).count()
            lsq10d = lsq10d['new'].mean()
        df = df.drop(['new'],axis=1,errors='ignore')
        out.extend([lsq10d])

    out.insert(0,int(df["gage"][0]))
    out = pd.DataFrame(out).transpose()
    out.columns = columns
    return out


In [100]:
def calcLowFlow(df):
    import warnings
    warnings.filterwarnings("ignore")
    '''
    Read in data files for gages and calculate number of annual low flow days
    '''
    

    stat = "LowFlow"
    columns = ["gage","VIC_"+stat,"PRMS_"+stat,"NWM2d0_"+stat,"NWM2d1_"+stat,"NWM3"+stat,"NWIS_"+stat]
    out = []

    for column in df[["streamflow_VIC","streamflow_PRMS",'streamflow_NWM2d0','streamflow_NWM2d1','streamflow_NWM3','streamflow_NWIS']]:

        #### Round columns to add zeros
        if column == "streamflow_NWIS":
            df[column] = df[column]/cfs_2_cms
            df[column] = np.round(df[column],1)
            df[column] = (df[column]*cfs_2_cms)
            # df.loc[(df[column]<=(df[column].quantile(0.25)))&(df[column]>0),"new"] = 1
            df.loc[(df[column]<=(df[column].quantile(0.25))),"new"] = 1
        else:
            # df.loc[(df[column]<=(df[column].quantile(0.25)))&(df[column]>(df[column].mean()*.1)),"new"] = 1
            df.loc[(df[column]<=(df[column].quantile(0.25))),"new"] = 1


        ## Calculate number of zero flow days by season
        lsq10d = df[['climate_year','new']]
        if df["new"].isnull().all():
            lsq10d = np.nan
        else:
            lsq10d = lsq10d.groupby(["climate_year"]).count()
            lsq10d = lsq10d['new'].mean()

        df = df.drop(['new'],axis=1,errors='ignore')
        out.extend([lsq10d])

    out.insert(0,int(df["gage"][0]))
    out = pd.DataFrame(out).transpose()
    out.columns = columns
    return out



In [98]:
i = 88

pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_HydroClass.csv")
epsil = 1e-4
df = readModelFile(i)
calcZeroFlow(df)

,gage,VIC_ZeroFlow,PRMS_ZeroFlow,NWM2d0_ZeroFlow,NWM2d1_ZeroFlow,NWM3ZeroFlow,NWIS_ZeroFlow
0,12201960.0,11.15,42.85,6.75,61.5,8.65,43.6


In [99]:
calcLowFlow(df)

,gage,VIC_LowFlow,PRMS_LowFlow,NWM2d0_LowFlow,NWM2d1_LowFlow,NWM3LowFlow,NWIS_LowFlow
0,12201960.0,46.8,3.95,73.65,40.65,83.75,44.55


In [101]:
calcLowFlow(df)

,gage,VIC_LowFlow,PRMS_LowFlow,NWM2d0_LowFlow,NWM2d1_LowFlow,NWM3LowFlow,NWIS_LowFlow
0,12201960.0,46.8,46.8,80.4,102.15,92.4,88.15


### Tests

In [42]:
i = 88

pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_HydroClass.csv")
epsil = 1e-4
df = readModelFile(i)
calcZeroFlow(df)

,gage,VIC_ZeroFlow,PRMS_ZeroFlow,NWM2d0_ZeroFlow,NWM2d1_ZeroFlow,NWM3ZeroFlow,NWIS_ZeroFlow
0,12201960.0,2.822785,10.848101,1.708861,15.56962,2.189873,11.037975


In [43]:
calcLowFlow(df)

,gage,VIC_LowFlow,PRMS_LowFlow,NWM2d0_LowFlow,NWM2d1_LowFlow,NWM3LowFlow,NWIS_LowFlow
0,12201960.0,11.848101,11.848101,20.35443,25.860759,23.392405,22.316456


### Tests

In [85]:
i = 88

pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_HydroClass.csv")
epsil = 1e-4
df = readModelFile(i)

stat = "LowFlow"
columns = ["gage","VIC_"+stat,"PRMS_"+stat,"NWM2d0_"+stat,"NWM2d1_"+stat,"NWM3"+stat,"NWIS_"+stat]
out = []

for column in df[["streamflow_VIC","streamflow_PRMS",'streamflow_NWM2d0','streamflow_NWM2d1','streamflow_NWM3','streamflow_NWIS']]:

    #### Round columns to add zeros
    if column == "streamflow_NWIS":
        df[column] = df[column]/cfs_2_cms
        df[column] = np.round(df[column],1)
        df[column] = (df[column]*cfs_2_cms)
        df.loc[(df[column]<=(df[column].quantile(0.25)))&(df[column]>0),"new"] = 1
    else:
        df.loc[(df[column]<=(df[column].quantile(0.25)))&(df[column]>(df[column].mean()*.1)),"new"] = 1

    ## Calculate number of zero flow days by season
    lsq10d = df[['climate_year','new']]
    if df["new"].isnull().all():
        lsq10d = np.nan
    else:
        lsq10d = lsq10d.groupby(["climate_year"]).count()
        # lsq10d = lsq10d['new'].mean()
        pass

    df = df.drop(['new'],axis=1,errors='ignore')
    out.extend([lsq10d])

In [86]:
tt = lsq10d.reset_index()
tt.new.mean()

44.55

In [87]:
lsq10d

,new
climate_year,
2002,0
2003,99
2004,33
2005,30
2006,96
2007,28
2008,47
2009,51
2010,6


In [88]:
i = 88

pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_HydroClass.csv")
epsil = 1e-4
df = readModelFile(i)

stat = "ZeroFlow"
columns = ["gage","VIC_"+stat,"PRMS_"+stat,"NWM2d0_"+stat,"NWM2d1_"+stat,"NWM3"+stat,"NWIS_"+stat]
out = []

for column in df[["streamflow_VIC","streamflow_PRMS",'streamflow_NWM2d0','streamflow_NWM2d1','streamflow_NWM3','streamflow_NWIS']]:
    
    #### Round columns to nearest 0.1 cfs, therefore anything less than 0.05 cfs is q=0
    #### Round columns to add zeros
    if column == "streamflow_NWIS":
        df[column] = df[column]/cfs_2_cms
        df[column] = np.round(df[column],1)
        df.loc[df[column]==0,"new"] = 1
    else:
        df.loc[df[column]<=(df[column].mean()*.1),"new"] = 1

    ## Calculate number of zero flow days by season
    lsq10d = df[['climate_year','new']]
    # if df["new"].isnull().all():
    #     lsq10d = np.nan
    # else:
    #     lsq10d = lsq10d.groupby(["climate_year"]).count()
    #     # lsq10d = lsq10d['new'].mean()

    # df = df.drop(['new'],axis=1,errors='ignore')
    # out.extend([lsq10d])


In [62]:
lsq10d = lsq10d.reset_index()
lsq10d['new'].mean()

11.037974683544304

In [75]:
lsq10d.describe()

,climate_year,new
count,7031.000000,2061.0
mean,2011.624947,1.0
std,5.569706,0.0
min,2002.000000,1.0
25%,2007.000000,1.0
50%,2012.000000,1.0
75%,2016.000000,1.0
max,2021.000000,1.0


In [90]:
lsq10d.groupby(["climate_year"]).count()

,new
climate_year,
2002,22
2003,169
2004,146
2005,73
2006,96
2007,143
2008,119
2009,102
2010,137
